In [ ]:
!pip install tensorflow
!pip install pytorch
!pip install torch
!pip install transformers
!pip install evaluate
!pip install pickle

In [48]:
import transformers
import pandas as pd
import os
from collections import defaultdict
import re

In [58]:
reports = {}

for file in os.listdir('./output/classified_sentences'):
    if file == ".DS_Store":
        continue
    df = pd.read_csv(f'./output/classified_sentences/{file}')
    df = df.rename(columns={'Unnamed: 0': 'Line'})
    # pick out ticker
    ticker = file.split('_')[0]
    reports[ticker] = df

In [59]:
# Topics as keys, and a list of all sentences classified as that topic as values

'''
    Todo: we need to change this.
    1. Check if mapped criteria topic 1 is 'NON-ESG'
        - if it is, continue (skip iteration)
    2. Check if mapped criteria topic 1 is empty string
        - if it is, check mapped criteria 2 (repeat step 1,2,3)
    3. Check if mapped criteria topic 1 has criteria topic
        - if there is 1 criteria topic, append sentence to corresponding topic and continue (next iteration)
        - if there are multiple criteria topics, append sentence to all corresponding topics and continue (next iteration)
'''

"\n    Todo: we need to change this.\n    1. Check if mapped criteria topic 1 is 'NON-ESG'\n        - if it is, continue (skip iteration)\n    2. Check if mapped criteria topic 1 is empty string\n        - if it is, check mapped criteria 2 (repeat step 1,2,3)\n    3. Check if mapped criteria topic 1 has criteria topic\n        - if there is 1 criteria topic, append sentence to corresponding topic and continue (next iteration)\n        - if there are multiple criteria topics, append sentence to all corresponding topics and continue (next iteration)\n"

In [60]:
from ast import literal_eval
topics_by_company = {}

for company, data in reports.items():
    topics_by_company[company] = {}
    for index, row in data.iterrows():
        mapped_criteria_topic_1 = literal_eval(row['Mapped Criteria Topic 1'])
        mapped_criteria_topic_2 = literal_eval(row['Mapped Criteria Topic 2'])
        mapped_criteria_topic_3 = literal_eval(row['Mapped Criteria Topic 3'])
        sentence = row['Sentence']
        if mapped_criteria_topic_1[0] != 'NON-ESG':
            if mapped_criteria_topic_1[0] != []:
                for i in  mapped_criteria_topic_1[0]:
                    if i not in topics_by_company[company]:
                        topics_by_company[company][i] = {}
                    topics_by_company[company][i][sentence] = mapped_criteria_topic_1[1]
            else:
                if mapped_criteria_topic_2[0] != 'NON-ESG':
                    if mapped_criteria_topic_2[0] != []:
                        for i in  mapped_criteria_topic_2[0]:
                            if i not in topics_by_company[company]:
                                topics_by_company[company][i] = {}
                            topics_by_company[company][i][sentence] = mapped_criteria_topic_2[1]
                    else:
                        if mapped_criteria_topic_3[0] != 'NON-ESG':
                            if mapped_criteria_topic_3[0] != []:
                                for i in  mapped_criteria_topic_3[0]:
                                    if i not in topics_by_company[company]:
                                        topics_by_company[company][i] = {}
                                    topics_by_company[company][i][sentence] = mapped_criteria_topic_3[1]       

In [62]:
# Sort sentences by topic according to probability
for company, data in topics_by_company.items():
    for topic, sentence_prob in data.items():
        sorted_sentence_prob = dict(sorted(sentence_prob.items(), key = lambda x: x[1], reverse=True))
        topics_by_company[company][topic] = sorted_sentence_prob

In [63]:
company_topic_sentences = {}
for company, data in topics_by_company.items():
    company_topic_sentences[company] = {}
    for topic, sentences in data.items():
        if topic not in company_topic_sentences[company]:
            company_topic_sentences[company][topic] = []
        for sentence, prob in sentences.items():
            company_topic_sentences[company][topic].append(sentence)

### Facebook-BART-CNN Model for summarization
- Use `pipeline` from Huggingface's `transformer` library
- For each company, save the generated summaries under 1 csv file under `Generated Summaries/fina/...`

In [68]:
if not os.path.exists("Generated Summaries/final"):
    os.makedirs("Generated Summaries/final")

In [69]:
from transformers import pipeline
import nltk
from nltk.tokenize import sent_tokenize

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Helper function to split the text into segments
def split_in_segments(text):
    tokens = 0
    mystring = list()
    segments = []
    text = re.sub(r'(?<=\w)(\.)', r' \1', text)
    for sent in sent_tokenize(text):
        newtokens = len(sent.split())
        if tokens + newtokens > 500:
            segments.append(" ".join(mystring))
            mystring = []
            mystring.append(str(sent).strip())
            tokens = newtokens
        elif newtokens > 500:
            segments.extend([str(sent)[i:i+500] for i in range(0, len(str(sent)), 500)])
        else:
            tokens += newtokens
            mystring.append(str(sent).strip())
        
    if mystring:
        segments.append(" ".join(mystring))

    return segments

# Helper Function to summarize text based on input text
def summarize_text(input_text):
    segments = split_in_segments(input_text)
    summaries = []
    for seg in segments:
        summary = summarizer(seg, truncation = True, max_length=100, min_length=30, do_sample=False)
        summaries.extend(summary)
    return summaries

#### Input used for summarization : Top 20 Sentences

In [70]:
company_summary_sentences = {}

for company, data in company_topic_sentences.items():
    print(f"Summarizing {company}")
    output = {}

    for topic, sentences in data.items():
        print(f"### Topic: {topic} ###")

        input_sentences = " ".join(sentences[:20]) # Top 20 sentences
        reference_summary = sentences[:40] # reference summary don't change
        summary_list = summarize_text(input_sentences)
        combined_summary = summary_list[0]['summary_text']
        for summary in summary_list[1:]:
            combined_summary += summary['summary_text']
        output[topic] = (combined_summary, " ".join(reference_summary))
    if company not in company_summary_sentences:
        company_summary_sentences[company] = {}
    company_summary_sentences[company] = output

    output_df = pd.DataFrame.from_dict(output, orient='index')
    output_df.to_csv(f"Generated Summaries/final/{company}_top20_summaries.csv")

Summarizing DHR
### Topic: Human Capital Development ###
### Topic: Innovation Management ###
### Topic: Operational Eco-Efficiency ###
### Topic: Talent Attraction & Retention ###
### Topic: Climate Strategy ###
### Topic: Business Ethics ###
### Topic: Risk & Crisis Management ###
### Topic: Supply Chain Management ###
Summarizing ABT
### Topic: Talent Attraction & Retention ###
### Topic: Supply Chain Management ###
### Topic: Business Ethics ###


Your max_length is set to 100, but you input_length is only 89. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)


### Topic: Human Capital Development ###
### Topic: Innovation Management ###
### Topic: Health Outcome Contribution ###
### Topic: Product Quality & Recall Management ###
### Topic: Operational Eco-Efficiency ###
Summarizing INTC
### Topic: Product Stewardship ###


Your max_length is set to 100, but you input_length is only 17. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=8)


### Topic: Climate Strategy ###
### Topic: Environmental Policy & Management Systems ###
### Topic: Talent Attraction & Retention ###
### Topic: Operational Eco-Efficiency ###
### Topic: Supply Chain Management ###


...

### Topic: Network Reliability ###


Your max_length is set to 100, but you input_length is only 88. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)


### Topic: Operational Eco-Efficiency ###
### Topic: Risk & Crisis Management ###
### Topic: Human Capital Development ###
### Topic: Biodiversity ###
Summarizing BAC
### Topic: Talent Attraction & Retention ###
### Topic: Financial Inclusion ###
### Topic: Human Capital Development ###


Your max_length is set to 100, but you input_length is only 43. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=21)


### Topic: Risk & Crisis Management ###
### Topic: Decarbonization Strategy ###
### Topic: Corporate Governance ###
### Topic: Business Ethics ###
### Topic: Sustainable Finance ###
### Topic: Climate Strategy ###


In [ ]:
company_summary_sentences = {}

for company, data in company_topic_sentences.items():
    print(f"Summarizing {company}")
    output = {}

    for topic, sentences in data.items():
        print(f"### Topic: {topic} ###")

        input_sentences = " ".join(sentences[:30]) # Top 30 sentences
        reference_summary = sentences[:40] # reference summary don't change
        summary_list = summarize_text(input_sentences)
        combined_summary = summary_list[0]['summary_text']
        for summary in summary_list[1:]:
            combined_summary += summary['summary_text']
        output[topic] = (combined_summary, " ".join(reference_summary))
    if company not in company_summary_sentences:
        company_summary_sentences[company] = {}
    company_summary_sentences[company] = output

    output_df = pd.DataFrame.from_dict(output, orient='index')
    output_df.to_csv(f"Generated Summaries/final/{company}_top30_summaries.csv")

#### Input used for summarization : Random 20/30 Sentences

In [71]:
import random
random.seed(88)

company_summary_sentences = {}

for company, data in company_topic_sentences.items():
    print(f"Summarizing {company}")
    output = {}

    for topic, sentences in data.items():
        print(f"### Topic: {topic} ###")

        input_sentences = sentences.copy()
        random.shuffle(input_sentences)
        input_sentences = " ".join(input_sentences[:20])

        reference_summary = sentences[:40] # reference summary don't change
        summary_list = summarize_text(input_sentences)
        combined_summary = summary_list[0]['summary_text']
        for summary in summary_list[1:]:
            combined_summary += summary['summary_text']
        output[topic] = (combined_summary, " ".join(reference_summary))
    if company not in company_summary_sentences:
        company_summary_sentences[company] = {}
    company_summary_sentences[company] = output

    output_df = pd.DataFrame.from_dict(output, orient='index')
    output_df.to_csv(f"Generated Summaries/final/{company}_random20_summaries.csv")

Summarizing DHR
### Topic: Human Capital Development ###
### Topic: Innovation Management ###
### Topic: Operational Eco-Efficiency ###
### Topic: Talent Attraction & Retention ###
### Topic: Climate Strategy ###
### Topic: Business Ethics ###
### Topic: Risk & Crisis Management ###
### Topic: Supply Chain Management ###
Summarizing ABT
### Topic: Talent Attraction & Retention ###
### Topic: Supply Chain Management ###
### Topic: Business Ethics ###
### Topic: Human Capital Development ###
### Topic: Innovation Management ###
### Topic: Health Outcome Contribution ###


Your max_length is set to 100, but you input_length is only 58. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)


### Topic: Product Quality & Recall Management ###
### Topic: Operational Eco-Efficiency ###
Summarizing INTC
### Topic: Product Stewardship ###
### Topic: Climate Strategy ###
### Topic: Environmental Policy & Management Systems ###
### Topic: Talent Attraction & Retention ###
### Topic: Operational Eco-Efficiency ###
### Topic: Supply Chain Management ###
### Topic: Human Capital Development ###
### Topic: Innovation Management ###
### Topic: Product Quality & Recall Management ###
Summarizing BMY
### Topic: Business Ethics ###
### Topic: Human Capital Development ###
### Topic: Innovation Management ###
### Topic: Access to Healthcare ###


Your max_length is set to 100, but you input_length is only 22. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Your max_length is set to 100, but you input_length is only 3. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=1)


### Topic: Talent Attraction & Retention ###


...

### Topic: Innovation Management ###
### Topic: Talent Attraction & Retention ###
### Topic: Information Security Cybersecurity & System Availability ###
### Topic: Operational Eco-Efficiency ###


Your max_length is set to 100, but you input_length is only 60. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=30)


### Topic: Climate Strategy ###
Summarizing LIN
### Topic: Human Capital Development ###
### Topic: Innovation Management ###
### Topic: Climate Strategy ###
### Topic: Operational Eco-Efficiency ###
### Topic: Product Stewardship ###
### Topic: Talent Attraction & Retention ###
### Topic: Business Ethics ###
### Topic: Water Related Risks ###
### Topic: Occupational Health & Safety ###
Summarizing CSCO
### Topic: Environmental Policy & Management Systems ###
### Topic: Human Capital Development ###
### Topic: Innovation Management ###
### Topic: Talent Attraction & Retention ###
### Topic: Information Security Cybersecurity & System Availability ###
### Topic: Supply Chain Management ###
### Topic: Operational Eco-Efficiency ###
### Topic: Product Stewardship ###
Summarizing VZ
### Topic: Climate Strategy ###
### Topic: Information Security Cybersecurity & System Availability ###
### Topic: Network Reliability ###


Your max_length is set to 100, but you input_length is only 46. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)


### Topic: Operational Eco-Efficiency ###
### Topic: Risk & Crisis Management ###
### Topic: Human Capital Development ###
### Topic: Biodiversity ###
Summarizing BAC
### Topic: Talent Attraction & Retention ###
### Topic: Financial Inclusion ###
### Topic: Human Capital Development ###
### Topic: Risk & Crisis Management ###
### Topic: Decarbonization Strategy ###
### Topic: Corporate Governance ###
### Topic: Business Ethics ###
### Topic: Sustainable Finance ###
### Topic: Climate Strategy ###


In [ ]:
import random
random.seed(88)

company_summary_sentences = {}

for company, data in company_topic_sentences.items():
    print(f"Summarizing {company}")
    output = {}

    for topic, sentences in data.items():
        print(f"### Topic: {topic} ###")

        input_sentences = sentences.copy()
        random.shuffle(input_sentences)
        input_sentences = " ".join(input_sentences[:30])

        reference_summary = sentences[:40] # reference summary don't change
        summary_list = summarize_text(input_sentences)
        combined_summary = summary_list[0]['summary_text']
        for summary in summary_list[1:]:
            combined_summary += summary['summary_text']
        output[topic] = (combined_summary, " ".join(reference_summary))
    if company not in company_summary_sentences:
        company_summary_sentences[company] = {}
    company_summary_sentences[company] = output

    output_df = pd.DataFrame.from_dict(output, orient='index')
    output_df.to_csv(f"Generated Summaries/final/{company}_random30_summaries.csv")

### Evaluation with Rouge-N from Huggingface's Evaluate
https://huggingface.co/spaces/evaluate-metric/rouge

### Comparing performance
1. Top 20 sentences as input
2. Top 30 sentences as input
3. Random 20 sentences
4. Random 30 sentences

Reference Summary -> Top 40 sentences

In [12]:
# Helper Function to compute ROUGE-N scores from Huggingface's evaluate library
import evaluate
def evaluateRouge(output):
    output_results = {}
    rouge = evaluate.load('rouge')
    for idx, row in output.iterrows():
        topic = row['Unnamed: 0']
        generated = [row[1]]
        reference = [row[2]]
        results = rouge.compute(predictions=generated, references = reference)
        output_results[topic] = results
    return output_results

In [ ]:
# Helper Function to calculate the average ROUGE-N score 
def averageEvaluation(results):
    output_dict = {
        'rouge1': 0,
        'rouge2': 0,
        'rougeL': 0,
        'rougeLsum': 0
    }
    for topic, rougeData in results.items():
        output_dict['rouge1'] = sum(val['rouge1'] for val in results.values())/len(results)
        output_dict['rouge2'] = sum(val['rouge2'] for val in results.values())/len(results)
        output_dict['rougeL'] = sum(val['rougeL'] for val in results.values())/len(results)
        output_dict['rougeLsum'] = sum(val['rougeLsum'] for val in results.values())/len(results)
    return output_dict

In [26]:
# For each company, compute average ROUGE-N scores across all generated summaries
import os
generated_summaries_path = "./Generated Summaries/final/"

evaluation_results = {}

for filename in os.listdir(generated_summaries_path):
    summaries = pd.read_csv(generated_summaries_path + filename)
    summaries_result = evaluateRouge(summaries)
    company = filename.split("_")[0]
    input_type = filename.split("_")[1]
    if input_type not in evaluation_results:
        evaluation_results[input_type] = {}
    evaluation_results[input_type][company] = averageEvaluation(summaries_result)    

In [42]:
# Save evaluation data as pickle file to reopen for review
import pickle

save_file = False # Set as true if you want to save the evaluation_results dictionary as pickle file

if save_file: # save evaluation results as pickle file
    file_to_save = open('Evaluation_Result', 'ab')
    pickle.dump(evaluation_results, file_to_save)
    save_file.close()
else: # Set as false to open the file
    saved_file = open('Evaluation_Result', 'rb')
    evaluation_results = pickle.load(saved_file)

In [41]:
# Calculate Average ROUGE-N scores across types of inputs (Top20/Top30/Random20/Random30)
show_evaluation = {}
for k,v in evaluation_results.items():
    show_evaluation[k] = {
        'rouge1': 0,
        'rouge2': 0,
        'rougeL': 0,
        'rougeLsum': 0
    }

    show_evaluation[k]['rouge1'] = sum(val['rouge1'] for val in v.values())/len(v)
    show_evaluation[k]['rouge2'] = sum(val['rouge2'] for val in v.values())/len(v)
    show_evaluation[k]['rougeL'] = sum(val['rougeL'] for val in v.values())/len(v)
    show_evaluation[k]['rougeLsum'] = sum(val['rougeLsum'] for val in v.values())/len(v)

pd.DataFrame.from_dict(show_evaluation, orient='index').sort_index()

,rouge1,rouge2,rougeL,rougeLsum
random20,0.225192,0.200951,0.172540,0.172540
random30,0.251613,0.222382,0.181262,0.181262
top20,0.218464,0.206740,0.197760,0.197760
top30,0.246358,0.232508,0.223546,0.223546
